# Análisis de tweets
Este notebook analiza la toxicidad de los tweets que nos provee el dataset *dataset_newtral_ucm.csv* según las columnas *toxicity* y *degree_predicted*. Podremos ver de manera ordenada la toxicidad media de distintos partidos, políticos y palabras.



In [1]:
import pandas as pd
import numpy as np

In [2]:
!python --version

Python 3.7.12


In [3]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  df = pd.read_csv('/content/drive/Shareddrives/ETICA/dataset_newtral_ucm.csv')
else:
  df = pd.read_csv('../data/dataset_newtral_ucm.csv')

Mounted at /content/drive


In [5]:
df.head(5)

,twitter_id,text,toxicity,sev_toxicity,profanity,threat,degree1,degree2,degree3,degree_final,degree_predicted,degree_confidence,annotation_date,author_id,slug,twitter_url,party_slug,parliamentary_group_slug
0,1344806255798480896,Feliz 2021 #Adios2020 https://t.co/ULpRE0T2AI,0.012811,0.022777,0.010508,0.077519,0,0,0,0,0,0.969,2021-10-07,2849798849,jose-alberto-herrero-bono,https://twitter.com/herrerobono,partido-popular,gp
1,1344822586396602368,Empezar el año con #CachitosNochevieja no tien...,0.085995,0.068680,0.302721,0.087808,0,0,0,0,0,0.984,2021-10-07,88802757,begona-nasarre-oliva,https://twitter.com/Begonasarre,psoe,gs
2,1344934218145669120,Llegará un momento que los miembros de este Go...,0.478526,0.122059,0.026271,0.513052,2,2,1,2,2,0.868,2021-10-07,593952938,juan-luis-steegmann-olmedillas,https://twitter.com/jlsteeg,vox,gvox
3,1344940715072221184,La frase “ Falsa sensación de seguridad” viene...,0.309307,0.040751,0.009823,0.122409,2,1,1,1,2,0.557,2021-08-25,593952938,juan-luis-steegmann-olmedillas,https://twitter.com/jlsteeg,vox,gvox
4,1344945712128266240,El Comisario político del Gobierno no descansa...,0.194637,0.072107,0.026215,0.249154,2,2,2,2,2,0.964,2021-10-07,1197128623364423680,macarena-montesinos-de-miguel,https://twitter.com/MackMontesinos,partido-popular,gp


In [6]:
#La lista de partidos a los que están afiliados los distintos políticos que twittean
partidos = df.party_slug.unique()
partidos = partidos[~pd.isna(partidos)]
print('Partidos: ', partidos)

Partidos:  ['partido-popular' 'psoe' 'vox' 'ciudadanos' 'podemos' 'en-comu-podem'
 'catalunya-en-comu' 'en-marea' 'sin-partido' 'izquierda-unida'
 'unidas-podemos']


##Toxicidad de los distintos partidos

In [7]:
### Calcular toxicidad 'media' de cada partido según sus tweets ###

tox_media = {i : 0 for i in partidos} # Toxicidad "media" de cada partido

for p in partidos:
    toxicity = tox_media.get(p)
    i = 0
    for idx, row in df[df['party_slug'] == p].iterrows():
        toxicity += row.toxicity #Usamos columna 'toxicity'
        i += 1
    
    tox_media.update({p: toxicity/i})

In [8]:
#Parece que los políticos de VOX twittean de manera más tóxica según esta columna
sorted(tox_media.items(), key=lambda item: item[1], reverse=True)

[('vox', 0.17254207772706207),
 ('catalunya-en-comu', 0.14572348429824114),
 ('ciudadanos', 0.13457874323427005),
 ('podemos', 0.12306653355605846),
 ('partido-popular', 0.1121905007956817),
 ('sin-partido', 0.08940209548297107),
 ('unidas-podemos', 0.08462033266705205),
 ('psoe', 0.06439602086209517),
 ('izquierda-unida', 0.0621266634409091),
 ('en-marea', 0.043359509957740607),
 ('en-comu-podem', -0.36142766800000004)]

In [9]:
### Calcular degree 'medio' de cada partido según sus tweets ###

deg_medio = {i : 0 for i in partidos} # Toxicidad "media" de cada partido

for p in partidos:
    toxicity = deg_medio.get(p)
    i = 0
    for idx, row in df[df['party_slug'] == p].iterrows():
        toxicity += row.degree_predicted#Usamos columna 'degree_predicted'
        i += 1
    
    deg_medio.update({p: toxicity/i})

In [10]:
#Parece que los políticos de VOX twittean de manera más tóxica según esta columna
sorted(deg_medio.items(), key=lambda item: item[1], reverse=True)

[('vox', 0.6836935166994106),
 ('catalunya-en-comu', 0.6155778894472361),
 ('partido-popular', 0.5689404934687954),
 ('ciudadanos', 0.48444976076555024),
 ('podemos', 0.4678832116788321),
 ('sin-partido', 0.2536231884057971),
 ('psoe', 0.1823877479986077),
 ('izquierda-unida', 0.10606060606060606),
 ('unidas-podemos', 0.10404624277456648),
 ('en-comu-podem', 0.06666666666666667),
 ('en-marea', 0.029288702928870293)]

In [11]:
#nº tweets con toxicidad "máxima" por partido utilizando la columna 'degree_predicted'
for p in partidos:
    print(p, np.count_nonzero(df[df['party_slug'] == p]['degree_predicted'] == 2))

partido-popular 467
psoe 150
vox 477
ciudadanos 243
podemos 214
en-comu-podem 0
catalunya-en-comu 80
en-marea 1
sin-partido 15
izquierda-unida 1
unidas-podemos 3


##Toxicidad de los distintos políticos
En esta sección, se ordenarán de mas a menos tóxicos, los políticos de todos los partidos. 

In [12]:
#Dataframe político - partido
politicos_part_df = df.loc[:, df.columns.intersection(['slug','party_slug',])]
politicos_part_df = politicos_part_df.drop_duplicates().dropna()

In [19]:
#Así están todos los políticos ordenados por partido de mayor a menor toxicidad por ambos algoritmos (columnas 'toxicity' y 'degree_predicted')
iterator = 0
for pa in partidos:
  tox_media_ = {i : 0 for i in politicos_part_df.loc[df['party_slug'] == pa]['slug']}
  deg_medio_ = {i : 0 for i in politicos_part_df.loc[df['party_slug'] == pa]['slug']}
  for p in politicos_part_df.loc[df['party_slug'] == pa]['slug']:
    toxicityg = tox_media_.get(p)
    toxicity = deg_medio_.get(p)
    i = 0
    for idx, row in df[df['slug'] == p].iterrows():
        toxicityg += row.toxicity
        toxicity += row.degree_predicted
        i += 1
    
    tox_media_.update({p: toxicityg/i})
    deg_medio_.update({p: toxicity/i})
  if iterator == 0: 
    tox_media_pp = sorted(tox_media_.items(), key=lambda x: x[1], reverse=True)
    deg_medio_pp = sorted(deg_medio_.items(), key=lambda x: x[1], reverse=True)
  elif iterator == 1:
    tox_media_psoe = sorted(tox_media_.items(), key=lambda x: x[1], reverse=True)
    deg_medio_psoe = sorted(deg_medio_.items(), key=lambda x: x[1], reverse=True)
  elif iterator == 2:
    tox_media_vox = sorted(tox_media_.items(), key=lambda x: x[1], reverse=True)
    deg_medio_vox = sorted(deg_medio_.items(), key=lambda x: x[1], reverse=True)
  elif iterator == 3:
    tox_media_cs = sorted(tox_media_.items(), key=lambda x: x[1], reverse=True)
    deg_medio_cs = sorted(deg_medio_.items(), key=lambda x: x[1], reverse=True)
  elif iterator == 4:
    tox_media_pdms = tox_media_
    deg_medio_pdms = deg_medio_
  elif iterator == 5:
    tox_media_ecp = sorted(tox_media_.items(), key=lambda x: x[1], reverse=True)
    deg_medio_ecp = sorted(deg_medio_.items(), key=lambda x: x[1], reverse=True)
  elif iterator == 6:
    tox_media_cec = sorted(tox_media_.items(), key=lambda x: x[1], reverse=True)
    deg_medio_cec = sorted(deg_medio_.items(), key=lambda x: x[1], reverse=True)
  elif iterator == 7:
    tox_media_em = sorted(tox_media_.items(), key=lambda x: x[1], reverse=True)
    deg_medio_em = sorted(deg_medio_.items(), key=lambda x: x[1], reverse=True)
  elif iterator == 8:
    tox_media_sp = sorted(tox_media_.items(), key=lambda x: x[1], reverse=True)
    deg_medio_sp = sorted(deg_medio_.items(), key=lambda x: x[1], reverse=True)
  elif iterator == 9:
    tox_media_iu = sorted(tox_media_.items(), key=lambda x: x[1], reverse=True)
    deg_medio_iu = sorted(deg_medio_.items(), key=lambda x: x[1], reverse=True)
  elif iterator == 10:
    tox_media_up = tox_media_
    deg_medio_up = deg_medio_
  iterator += 1


In [20]:
def unir_diccionarios(x, y):
    z = x.copy()   
    z.update(y)    
    return z

In [21]:
tox_media_podemos = sorted(unir_diccionarios(tox_media_pdms, tox_media_up).items(), key=lambda x: x[1], reverse=True)
deg_medio_podemos = sorted(unir_diccionarios(deg_medio_pdms, deg_medio_up).items(), key=lambda x: x[1], reverse=True)

###Políticos de Vox

In [22]:
#columna 'toxicity'
#El político que twittea de manera más tóxica dentro de Vox es Andrés Alberto Rodríguez Almeida
tox_media_vox

[('andres-alberto-rodriguez-almeida', 0.4172059),
 ('joaquin-robles-lopez', 0.3540057367407407),
 ('carla-toscano-de-balbin', 0.3043675460045112),
 ('santiago-abascal-conde', 0.27520808310312495),
 ('julio-utrilla-cano', 0.262878443),
 ('luis-gestoso-de-miguel', 0.25148037261904777),
 ('rodrigo-jimenez-revuelta', 0.24172074899999998),
 ('mireia-borras-pabon', 0.2172778154),
 ('manuel-mariscal-zabala', 0.21243887109090911),
 ('jose-maria-figaredo-alvarez-sala', 0.20994074400740736),
 ('antonio-salva-verd', 0.20138511),
 ('patricia-de-las-heras-fernandez', 0.1920542359727273),
 ('rocio-de-meer-mendez', 0.19106228085245902),
 ('juan-luis-steegmann-olmedillas', 0.19022899600292223),
 ('francisco-jose-alcaraz-martos', 0.18407546938064526),
 ('francisco-jose-contreras-pelaez', 0.17388685164593837),
 ('ignacio-garriga-vaz-de-concicao', 0.1615828),
 ('ines-maria-canizares-pacheco', 0.15948938917021274),
 ('carlos-jose-zambrano-garcia-raez', 0.1566225715),
 ('victor-gonzalez-coello-de-portugal'

In [23]:
#columna 'degree_predicted'
#El político que twittea de manera más tóxica dentro de Vox es Antonio Salvá Verd
deg_medio_vox

[('antonio-salva-verd', 2.0),
 ('santiago-abascal-conde', 1.140625),
 ('luis-gestoso-de-miguel', 1.0952380952380953),
 ('carla-toscano-de-balbin', 1.037593984962406),
 ('manuel-mariscal-zabala', 1.0),
 ('julio-utrilla-cano', 1.0),
 ('andres-alberto-rodriguez-almeida', 1.0),
 ('francisco-jose-alcaraz-martos', 0.9838709677419355),
 ('joaquin-robles-lopez', 0.9259259259259259),
 ('mireia-borras-pabon', 0.8),
 ('juan-luis-steegmann-olmedillas', 0.7922077922077922),
 ('rocio-de-meer-mendez', 0.7540983606557377),
 ('patricia-de-las-heras-fernandez', 0.7272727272727273),
 ('ines-maria-canizares-pacheco', 0.723404255319149),
 ('jose-maria-figaredo-alvarez-sala', 0.6666666666666666),
 ('pablo-saez-alonso-munumer', 0.6666666666666666),
 ('maria-magdalena-nevado-del-campo', 0.5925925925925926),
 ('ignacio-garriga-vaz-de-concicao', 0.5714285714285714),
 ('cristina-alicia-esteban-calonje', 0.5454545454545454),
 ('francisco-jose-contreras-pelaez', 0.5434173669467787),
 ('ivan-espinosa-de-los-montero

###Políticos del PP

In [24]:
#columna 'toxicity'
#El político que twittea de manera más tóxica dentro de PP es Edurne Uriarte Bengoechea
tox_media_pp

[('edurne-uriarte-bengoechea', 0.22633430675903615),
 ('beatriz-alvarez-fanjul', 0.22021356743396223),
 ('eloy-suarez-lamata', 0.21132094783636357),
 ('fernando-adolfo-gutierrez-diaz-de-otazu', 0.1943408706),
 ('oscar-clavell-lopez', 0.19189163),
 ('ana-maria-beltran-villalba', 0.18819559969090902),
 ('macarena-montesinos-de-miguel', 0.18325074737500002),
 ('jaime-eduardo-de-olano-vela', 0.17539583635599998),
 ('jose-angel-alonso-perez', 0.16911878433333336),
 ('antonio-pablo-gonzalez-terol', 0.16855722363750006),
 ('ana-belen-vazquez-blanco', 0.16172726193303172),
 ('pablo-casado-blanco', 0.16050226850514024),
 ('pablo-hispan-iglesias-de-ussel', 0.15430900573333334),
 ('sebastian-jesus-ledesma-martin', 0.14868497860000002),
 ('jesus-postigo-quintana', 0.14273375),
 ('carmen-riolobos-regadera', 0.1384714856610811),
 ('rosa-maria-romero-sanchez', 0.1325315579811321),
 ('pablo-montesinos-aguayo', 0.12988913414),
 ('mario-garces-sanagustin', 0.12942102816250003),
 ('concepcion-gamarra-rui

In [25]:
#columna 'degree_predicted'
#El político que twittea de manera más tóxica dentro de PP es Javier Bas Corugeira
deg_medio_pp

[('javier-bas-corugeira', 2.0),
 ('maria-valentina-martinez-ferro', 1.4),
 ('macarena-montesinos-de-miguel', 1.3125),
 ('pedro-navarro-lopez', 1.25),
 ('eloy-suarez-lamata', 1.0909090909090908),
 ('jesus-postigo-quintana', 1.0),
 ('ana-belen-vazquez-blanco', 0.9728506787330317),
 ('edurne-uriarte-bengoechea', 0.927710843373494),
 ('jose-angel-alonso-perez', 0.875),
 ('jaime-eduardo-de-olano-vela', 0.824),
 ('margarita-prohens-rigo', 0.8),
 ('sebastian-jesus-ledesma-martin', 0.8),
 ('antonio-pablo-gonzalez-terol', 0.7857142857142857),
 ('pablo-montesinos-aguayo', 0.72),
 ('ana-maria-beltran-villalba', 0.6909090909090909),
 ('pablo-casado-blanco', 0.6869158878504673),
 ('miguel-angel-castellon-rubio', 0.68),
 ('concepcion-gamarra-ruiz-clavijo', 0.6376811594202898),
 ('carmen-riolobos-regadera', 0.6216216216216216),
 ('eduardo-carazo-hermoso', 0.6129032258064516),
 ('pablo-hispan-iglesias-de-ussel', 0.6),
 ('fernando-adolfo-gutierrez-diaz-de-otazu', 0.6),
 ('elena-castillo-lopez', 0.57894

###Políticos del PSOE

In [27]:
#columna 'toxicity'
#El político que twittea de manera más tóxica dentro del PSOE es Zaida Cantera de Castro
tox_media_psoe

[('zaida-cantera-de-castro', 0.3768706151333333),
 ('david-serrada-pariente', 0.278770788),
 ('jose-zaragoza-alonso', 0.24925391955772366),
 ('maria-montserrat-garcia-chavarria', 0.18343568),
 ('maria-dolores-narvaez-bandera', 0.1652950585),
 ('daniel-vicente-viondi', 0.16217110829268297),
 ('laura-berja-vega', 0.159298829336),
 ('juan-luis-soto-burillo', 0.15831786922816907),
 ('susana-ros-martinez', 0.15352469266666668),
 ('beatriz-micaela-carrillo-de-los-reyes', 0.1534886995171429),
 ('juan-francisco-serrano-martinez', 0.14675571127014925),
 ('eva-patricia-bueno-campanario', 0.12227629276666664),
 ('marisol-sanchez-jodar', 0.12139918571428572),
 ('cristina-lopez-zamora', 0.12105260708421052),
 ('lidia-guinart-moreno', 0.12056053332359552),
 ('patxi-lopez-alvarez', 0.11570173455555555),
 ('mariana-de-gracia-canales-duque', 0.11183354812499999),
 ('maria-luz-martinez-seijo', 0.1112835742255319),
 ('esther-padilla-ruiz', 0.11043024006666666),
 ('jose-antonio-rodriguez-salas', 0.1033426

In [28]:
#columna 'degree_predicted'
#El político que twittea de manera más tóxica dentro del PSOE es Zaida Cantera de Castro
deg_medio_psoe

[('zaida-cantera-de-castro', 1.1),
 ('ariagona-gonzalez-perez', 1.0),
 ('herminio-rufino-sancho-iniguez', 1.0),
 ('sergio-gutierrez-prieto', 1.0),
 ('maria-inmaculada-oria-lopez', 1.0),
 ('jose-zaragoza-alonso', 0.9186991869918699),
 ('maria-dolores-narvaez-bandera', 0.75),
 ('montse-minguez-garcia', 0.75),
 ('german-renau-martinez', 0.5),
 ('sofia-hernanz-costa', 0.5),
 ('david-serrada-pariente', 0.5),
 ('daniel-vicente-viondi', 0.4634146341463415),
 ('sonia-guerra-lopez', 0.4444444444444444),
 ('marina-ortega-otero', 0.42857142857142855),
 ('juan-luis-soto-burillo', 0.4225352112676056),
 ('juan-francisco-serrano-martinez', 0.417910447761194),
 ('esther-padilla-ruiz', 0.4),
 ('cristina-lopez-zamora', 0.39473684210526316),
 ('carmen-andres-anon', 0.375),
 ('felipe-jesus-sicilia-alferez', 0.3333333333333333),
 ('lidia-guinart-moreno', 0.3258426966292135),
 ('carmen-banos-ruiz', 0.3125),
 ('raquel-pedraja-sainz', 0.2857142857142857),
 ('esther-pena-camarero', 0.2777777777777778),
 ('jose

###Políticos de Ciudadanos

In [29]:
#columna 'toxicity'
#El político que twittea de manera más tóxica dentro de Cs es Miguel Ángel Gutiérrez Vivas
tox_media_cs

[('miguel-angel-gutierrez-vivas', 0.156673834375),
 ('ines-arrimadas-garcia', 0.14885648482769798),
 ('guillermo-diaz-gomez', 0.13137282671685943),
 ('jose-maria-espejo-saavedra-conesa', 0.12374897945652173),
 ('maria-munoz-vidal', 0.12343674822951652),
 ('sara-gimenez-gimenez', 0.1229274938888889),
 ('juan-ignacio-lopez-bas-valero', 0.1128738132836364)]

In [30]:
#columna 'degree_predicted'
#El político que twittea de manera más tóxica dentro de Cs es Juan Ignacio López-Bas Valero
deg_medio_cs

[('juan-ignacio-lopez-bas-valero', 0.6363636363636364),
 ('maria-munoz-vidal', 0.5521628498727735),
 ('miguel-angel-gutierrez-vivas', 0.5),
 ('guillermo-diaz-gomez', 0.45950413223140496),
 ('ines-arrimadas-garcia', 0.45863309352517984),
 ('jose-maria-espejo-saavedra-conesa', 0.41304347826086957),
 ('sara-gimenez-gimenez', 0.2222222222222222)]

###Políticos de Unidas Podemos

In [31]:
#columna 'toxicity'
#El político que twittea de manera más tóxica dentro de Podemos es Pablo Echenique Robba
tox_media_podemos

[('pablo-echenique-robba', 0.2188177331741855),
 ('javier-sanchez-serna', 0.21447197421568623),
 ('pedro-antonio-honrubia-hurtado', 0.21120378246511626),
 ('martina-velarde-gomez', 0.16993583902857143),
 ('txema-guijarro-garcia', 0.16701568575),
 ('irene-maria-montero-gil', 0.1523150716402598),
 ('maria-del-carmen-pita-cardenes', 0.142968169125),
 ('antonia-jover-diaz', 0.11868670638235296),
 ('roberto-uriarte-torrealday', 0.11524275434782608),
 ('maria-gloria-elizo-serrano', 0.11315106449047616),
 ('alberto-rodriguez-rodriguez', 0.10102757823076923),
 ('isabel-franco-carmona', 0.09750853758461539),
 ('juan-antonio-delgado-ramos', 0.0946046342173913),
 ('juan-antonio-lopez-de-uralde-garmendia', 0.08672587237795915),
 ('ione-belarra-urteaga', 0.08462033266705205),
 ('lucia-munoz-dalda', 0.07970724481521742),
 ('rafael-mayoral-perales', 0.075645962),
 ('roser-maestro-moliner', 0.057785342),
 ('pilar-garrido-gutierrez', 0.0185830252),
 ('anton-gomez-reino-varela', 0.004461921580246917),
 

In [32]:
#columna 'degree_predicted'
#El político que twittea de manera más tóxica dentro de Podemos es Txema Guijarro García
deg_medio_podemos

[('txema-guijarro-garcia', 1.25),
 ('pablo-echenique-robba', 0.8421052631578947),
 ('pedro-antonio-honrubia-hurtado', 0.7906976744186046),
 ('martina-velarde-gomez', 0.6285714285714286),
 ('rafael-mayoral-perales', 0.5714285714285714),
 ('javier-sanchez-serna', 0.5686274509803921),
 ('juan-antonio-delgado-ramos', 0.5652173913043478),
 ('maria-mercedes-perez-merino', 0.5384615384615384),
 ('anton-gomez-reino-varela', 0.5308641975308642),
 ('sofia-fernandez-castanon', 0.5217391304347826),
 ('maria-del-carmen-pita-cardenes', 0.5),
 ('antonia-jover-diaz', 0.4411764705882353),
 ('alberto-rodriguez-rodriguez', 0.38461538461538464),
 ('maria-gloria-elizo-serrano', 0.3238095238095238),
 ('marisa-saavedra-munoz', 0.2857142857142857),
 ('lucia-munoz-dalda', 0.21739130434782608),
 ('roberto-uriarte-torrealday', 0.21739130434782608),
 ('juan-antonio-lopez-de-uralde-garmendia', 0.1510204081632653),
 ('pilar-garrido-gutierrez', 0.15),
 ('ione-belarra-urteaga', 0.10404624277456648),
 ('irene-maria-mo

###Políticos de En Comú Podem

In [33]:
#columna 'toxicity'
tox_media_ecp

[('laura-lopez-dominguez', -0.36142766800000004)]

In [34]:
#columna 'degree_predicted'
deg_medio_ecp

[('laura-lopez-dominguez', 0.06666666666666667)]

###Políticos de Catalunya En Comú


In [35]:
#columna 'toxicity'
#El político que twittea de manera más tóxica dentro de Catalunya En Comú es Gerardo Pisarello Prados
tox_media_cec

[('gerardo-pisarello-prados', 0.20099022740336137),
 ('maria-del-mar-garcia-puig', 0.14526270310526318),
 ('jaume-asens-llodra', 0.13622082410621125),
 ('joan-mena-arca', 0.11695249486097557),
 ('aina-vidal-saez', -0.01185523111764707)]

In [36]:
#columna 'degree_predicted'
#El político que twittea de manera más tóxica dentro de Catalunya En Comú es Jaume Asens i Llodrà
deg_medio_cec

[('jaume-asens-llodra', 0.6894409937888198),
 ('gerardo-pisarello-prados', 0.6722689075630253),
 ('joan-mena-arca', 0.5365853658536586),
 ('aina-vidal-saez', 0.5294117647058824),
 ('maria-del-mar-garcia-puig', 0.05263157894736842)]

###Políticos de En Marea

In [37]:
#columna 'toxicity'
tox_media_em

[('yolanda-diaz-perez', 0.043359509957740607)]

In [38]:
#columna 'degree_predicted'
deg_medio_em

[('yolanda-diaz-perez', 0.029288702928870293)]

###Políticos de Izquierda Unida

In [39]:
#columna 'toxicity'
#El político que twittea de manera más tóxica dentro de Izquierda Unida es Miguel Ángel Bustamante Martín
tox_media_iu

[('miguel-angel-bustamante-martin', 0.075738369),
 ('alberto-garzon-espinosa', 0.05910183998333335)]

In [40]:
#columna 'degree_predicted'
#El político que twittea de manera más tóxica dentro de Izquierda Unida es Miguel Ángel Bustamante Martín
deg_medio_iu

[('miguel-angel-bustamante-martin', 0.16666666666666666),
 ('alberto-garzon-espinosa', 0.09259259259259259)]

###Políticos sin Partido

In [41]:
#columna 'toxicity'
#El político que twittea de manera más tóxica sin partido es María Victoria Rosell Aguilar
tox_media_sp

[('maria-victoria-rosell-aguilar', 0.23285156742857144),
 ('rafael-simancas-simancas', 0.18056464855),
 ('enrique-fernando-santiago-romero', 0.11699904978596493),
 ('jose-javier-izquierdo-roncero', 0.06358890262033899),
 ('elena-manez-rodriguez', 0.0201742094),
 ('maria-marrodan-funes', 0.0121286325),
 ('pilar-cancela-rodriguez', -0.018069713000000005),
 ('hector-illueca-ballester', -0.12229084042916666)]

In [42]:
#columna 'degree_predicted'
#El político que twittea de manera más tóxica sin patrtido es María Victoria Rosell Aguilar
deg_medio_sp

[('maria-victoria-rosell-aguilar', 0.8571428571428571),
 ('rafael-simancas-simancas', 0.5),
 ('enrique-fernando-santiago-romero', 0.2631578947368421),
 ('hector-illueca-ballester', 0.16666666666666666),
 ('jose-javier-izquierdo-roncero', 0.0847457627118644),
 ('elena-manez-rodriguez', 0.0),
 ('pilar-cancela-rodriguez', 0.0),
 ('maria-marrodan-funes', 0.0)]

### Toxicidad de todos los políticos independientemente de partido

In [45]:
#Todos los políticos ordenados de mayor a menor toxicidad independientemente de partido por ambos algoritmos
politicos_toxicidad_ord = sorted(dict(tox_media_pp + tox_media_psoe + tox_media_vox + tox_media_cs + tox_media_podemos + tox_media_ecp + tox_media_cec + tox_media_em + tox_media_sp + tox_media_iu).items(), key=lambda x: x[1], reverse=True)
politicos_gradomed_ord = sorted(dict(deg_medio_pp + deg_medio_psoe + deg_medio_vox + deg_medio_cs + deg_medio_podemos + deg_medio_ecp + deg_medio_cec + deg_medio_em + deg_medio_sp + deg_medio_iu).items(), key=lambda x: x[1], reverse=True)


In [46]:
#Miramos la toxicidad según 'toxicity' de todos los políticos de todos los partidos
#El político más tóxico es Andres Alberto Rodríguez Almeida
politicos_toxicidad_ord

[('andres-alberto-rodriguez-almeida', 0.4172059),
 ('zaida-cantera-de-castro', 0.3768706151333333),
 ('joaquin-robles-lopez', 0.3540057367407407),
 ('carla-toscano-de-balbin', 0.3043675460045112),
 ('david-serrada-pariente', 0.278770788),
 ('santiago-abascal-conde', 0.27520808310312495),
 ('julio-utrilla-cano', 0.262878443),
 ('luis-gestoso-de-miguel', 0.25148037261904777),
 ('jose-zaragoza-alonso', 0.24925391955772366),
 ('rodrigo-jimenez-revuelta', 0.24172074899999998),
 ('maria-victoria-rosell-aguilar', 0.23285156742857144),
 ('edurne-uriarte-bengoechea', 0.22633430675903615),
 ('beatriz-alvarez-fanjul', 0.22021356743396223),
 ('pablo-echenique-robba', 0.2188177331741855),
 ('mireia-borras-pabon', 0.2172778154),
 ('javier-sanchez-serna', 0.21447197421568623),
 ('manuel-mariscal-zabala', 0.21243887109090911),
 ('eloy-suarez-lamata', 0.21132094783636357),
 ('pedro-antonio-honrubia-hurtado', 0.21120378246511626),
 ('jose-maria-figaredo-alvarez-sala', 0.20994074400740736),
 ('antonio-sa

In [47]:
#Miramos la toxicidad según 'degree_predicted' de todos los políticos de todos los partidos
#Los políticos más tóxicos son Javier Bas Corugeira y Antonio Salvá Verd
politicos_gradomed_ord

[('javier-bas-corugeira', 2.0),
 ('antonio-salva-verd', 2.0),
 ('maria-valentina-martinez-ferro', 1.4),
 ('macarena-montesinos-de-miguel', 1.3125),
 ('pedro-navarro-lopez', 1.25),
 ('txema-guijarro-garcia', 1.25),
 ('santiago-abascal-conde', 1.140625),
 ('zaida-cantera-de-castro', 1.1),
 ('luis-gestoso-de-miguel', 1.0952380952380953),
 ('eloy-suarez-lamata', 1.0909090909090908),
 ('carla-toscano-de-balbin', 1.037593984962406),
 ('jesus-postigo-quintana', 1.0),
 ('ariagona-gonzalez-perez', 1.0),
 ('herminio-rufino-sancho-iniguez', 1.0),
 ('sergio-gutierrez-prieto', 1.0),
 ('maria-inmaculada-oria-lopez', 1.0),
 ('manuel-mariscal-zabala', 1.0),
 ('julio-utrilla-cano', 1.0),
 ('andres-alberto-rodriguez-almeida', 1.0),
 ('francisco-jose-alcaraz-martos', 0.9838709677419355),
 ('ana-belen-vazquez-blanco', 0.9728506787330317),
 ('edurne-uriarte-bengoechea', 0.927710843373494),
 ('joaquin-robles-lopez', 0.9259259259259259),
 ('jose-zaragoza-alonso', 0.9186991869918699),
 ('jose-angel-alonso-per

##Diccionario de palabras tóxicas




In [48]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [49]:
# Según columna 'toxicity'

# tox_palabras = {i : (0,0) for i in dicc_palabras} # Toxicidad "media" de cada palabra
tox_palabras = {} #no utilizamos un diccionario con las palabras en español, solo nos interesan las palabras que salen en los tweets
for  idx,row in df.iterrows():
  for tw in row.text.split():
    if (not tw.startswith('https')) and (not tw.startswith('@')) and (not tw.startswith('#')):
      for c in "\"“!':,;-.$%&~#ºª()*+/\@¿?¡^`´0123456789|[]”–=_«»":
        tw = tw.replace(c, "")
      try: 
        toxicity = tox_palabras.get(tw)[0]
        freq = tox_palabras.get(tw)[1]
        toxicity += row.toxicity
        freq += 1
        tox_palabras.update({tw: (toxicity, freq)})
      except:
        tox_palabras.update({tw: (row.toxicity, 1)})
toxWords = {i : (tox_palabras.get(i)[0]/tox_palabras.get(i)[1],tox_palabras.get(i)[1]) for i in tox_palabras if (i not in stopwords.words('spanish'))}

In [50]:
sorted(toxWords.items(), key=lambda x: (x[1][0],x[1][1]), reverse=True)

[('Colonos', (0.94864684, 1)),
 ('Viven', (0.94864684, 1)),
 ('Bestias', (0.94864684, 1)),
 ('Insecticida', (0.94864684, 1)),
 ('ñordos', (0.94864684, 1)),
 ('Puta', (0.94864684, 1)),
 ('histérica', (0.94864684, 1)),
 ('Cerda', (0.94864684, 1)),
 ('Desgraciada', (0.94864684, 1)),
 ('Puntos', (0.94864684, 1)),
 ('First', (0.94864684, 1)),
 ('Ekain', (0.94604987, 1)),
 ('Perrino', (0.94604987, 1)),
 ('hospitalizado', (0.94604987, 1)),
 ('noa', (0.94604987, 1)),
 ('soldadas', (0.9427241, 1)),
 ('hispanistas', (0.9423530000000001, 1)),
 ('gilipollas', (0.9423530000000001, 1)),
 ('marqueses', (0.9423530000000001, 1)),
 ('elemental', (0.9423530000000001, 1)),
 ('jacobinofeministacomunistaespañol', (0.9419799999999999, 1)),
 ('delirios', (0.9419799999999999, 1)),
 ('corroe', (0.9419799999999999, 1)),
 ('peloteo', (0.9419799999999999, 1)),
 ('Mamporrero', (0.9419799999999999, 1)),
 ('frustrado', (0.9419799999999999, 1)),
 ('correlato', (0.93742704, 1)),
 ('unirte', (0.93742704, 1)),
 ('mentirt

In [52]:
# Según columna 'degree_predicted'

# tox_palabras = {i : (0,0) for i in dicc_palabras} # Toxicidad "media" de cada palabra
tox_palabras = {} #no utilizamos un diccionario con las palabras en español, solo nos interesan las palabras q salen en los tweets
for  idx,row in df.iterrows():
  for tw in row.text.split():
    if (not tw.startswith('https')) and (not tw.startswith('@')) and (not tw.startswith('#')):
      for c in "\"“!':,;-.$%&~#ºª()*+/\@¿?¡^`´0123456789|[]”–=_«»":
        tw = tw.replace(c, "")
      try: 
        toxicity = tox_palabras.get(tw)[0]
        freq = tox_palabras.get(tw)[1]
        toxicity += row.degree_predicted
        freq += 1
        tox_palabras.update({tw: (toxicity, freq)})
      except:
        tox_palabras.update({tw: (row.degree_predicted, 1)})
toxWords = {i : (tox_palabras.get(i)[0]/tox_palabras.get(i)[1],tox_palabras.get(i)[1]) for i in tox_palabras if (i not in stopwords.words('spanish'))}


In [53]:
sorted(toxWords.items(), key=lambda x: x[1], reverse=True)


[('indigno', (2.0, 12)),
 ('sinvergüenza', (2.0, 9)),
 ('socialcomunistarra', (2.0, 8)),
 ('mentiroso', (2.0, 8)),
 ('desvergüenza', (2.0, 6)),
 ('ajena', (2.0, 6)),
 ('traidores', (2.0, 6)),
 ('cobardes', (2.0, 5)),
 ('Galapagar', (2.0, 5)),
 ('adoctrinar', (2.0, 5)),
 ('gentuza', (2.0, 5)),
 ('Marlasca', (2.0, 5)),
 ('OBJ', (2.0, 5)),
 ('Incapaces', (2.0, 5)),
 ('fallido', (2.0, 4)),
 ('Soros', (2.0, 4)),
 ('sinvergüenzas', (2.0, 4)),
 ('irá', (2.0, 4)),
 ('inventado', (2.0, 4)),
 ('odiar', (2.0, 4)),
 ('incompetentes', (2.0, 4)),
 ('soporta', (2.0, 4)),
 ('homófobo', (2.0, 4)),
 ('bochorno', (2.0, 4)),
 ('cínicos', (2.0, 4)),
 ('jauría', (2.0, 4)),
 ('transfuguismo', (2.0, 4)),
 ('imputados', (2.0, 4)),
 ('Lara', (2.0, 4)),
 ('mintiendo', (2.0, 4)),
 ('estiércol', (2.0, 4)),
 ('chantajear', (2.0, 4)),
 ('🤥No', (2.0, 4)),
 ('👉🏻Quienes', (2.0, 4)),
 ('carcoma', (2.0, 3)),
 ('asaltaron', (2.0, 3)),
 ('Vomitivo', (2.0, 3)),
 ('hablen', (2.0, 3)),
 ('sacamos', (2.0, 3)),
 ('inmersión', (